In [1]:
# !pip install langchain_google_genai
# !pip install -U langchain-community
# !pip install faiss-cpu

In [2]:
import os
from IPython.display import display, Markdown
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough, RunnableMap

In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [4]:
def load_model(model_name="gemini-2.5-flash-lite"):
  llm = ChatGoogleGenerativeAI(model=model_name)
  return llm

In [5]:
model = load_model()

In [6]:
# import requests
# def get_image(url,filename,extension):
#   content = requests.get(url).content
#   with open(f'/content/{filename}.{extension}', 'wb') as f:
#     f.write(content)
#   image = Image.open(f"/content/{filename}.{extension}")
#   plt.imshow(image)
#   return image

In [7]:
# image = get_image("https://static.nike.com/a/images/t_PDP_1728_v1/f_auto,q_auto:eco/1705ca64-fbc8-4b79-a451-4ab77760c219/dunk-low-older-shoes-C7T1cx.png",
#                   "nike-shoes",
#                   "png")

In [8]:
prompt="give me summary of this image in 5 words"

In [9]:
message= HumanMessage(
    content=[
         {
            "type": "text",
            "text": prompt,
        },
        {

            "type": "image_url", "image_url": "https://static.nike.com/a/images/t_PDP_1728_v1/f_auto,q_auto:eco/1705ca64-fbc8-4b79-a451-4ab77760c219/dunk-low-older-shoes-C7T1cx.png",

        }
    ]
)

In [10]:
print(model.invoke([message]).content)

White and grey Nike Dunk low.


In [11]:
loader = TextLoader("/content/nike_shoes.txt")
text = loader.load()[0].page_content
print(text)

Nike Air Max Plus sneakers. They feature a brown upper with a black Nike Swoosh logo on the side and a visible Air Max unit in the heel. The sole is white.
Here are some more details about the Nike Air Max Plus:
Style: TN
Release date: January 1, 2017
Style code: 852630-300
Original retail price: $150 USD
The Air Max Plus, also known as the TN, is a popular Nike running shoe that was first released in 1998. It is known for its unique design, which includes a gradient upper, visible Air Max units, and a wavy outsole. The TN has been a popular shoe among sneakerheads and casual wearers alike for over two decades.
It features a brown upper with a black Swoosh logo and a white sole. The shoe is currently available for resale on the StockX marketplace for an average price of around $150 USD.
Nike Air Max Plus Store Location: "Kings Way, Kathmandu, Nepal".
\n\n\n



Nike Dunk Low Retro sneakers.
Here are some more details about the Nike Dunk Low Retro:
Style: Low-top
Release date: October 31

In [12]:
def get_text_chunks_langchain(text):
  text_splitter = CharacterTextSplitter(chunk_size=20, chunk_overlap=10)
  docs = [Document(page_content=x) for x in text_splitter.split_text(text)]
  return docs

In [13]:
docs = get_text_chunks_langchain(text)

In [14]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [15]:
vectorstore = FAISS.from_documents(docs,embedding=embeddings)

In [16]:
retriever=vectorstore.as_retriever()

In [17]:
retriever.invoke("Nike slide/sandal.")

[Document(id='dc260cce-ffa2-444d-a196-490ad959ddfb', metadata={}, page_content='Nike slide/sandal.\nThey appear to be the Nike Slide/Sandal, which is a unisex slide/sandal.\nHere are some of the features:\nSoft, one-piece upper: The upper is made of a soft, synthetic material that provides comfort and support.\nPhylon midsole: The midsole is made of Phylon, which provides cushioning and support.\nRubber outsole: The outsole is made of rubber for traction and durability.\nSwoosh logo: The Nike Swoosh logo is on the strap of the sandal.\nAvailable in a variety of colors: The Nike Benassi Solarsoft Sandal is available in a variety of colors, including black, white, and beige.\nNike off courte slides store location: "Bhaktapur, Nepal".'),
 Document(id='e2fd3177-f623-4c44-8bc1-5a370e2582a0', metadata={}, page_content='Nike Dunk Low Retro sneakers.\nHere are some more details about the Nike Dunk Low Retro:\nStyle: Low-top\nRelease date: October 31, 2020\nStyle code: DD1391-100\nOriginal reta

In [18]:
prompt = ChatPromptTemplate.from_messages([
    (
        "human",
        "You are given an image and some background context.\n\n"
        "Context:\n{context}\n\n"
        "Query: {query}\n\n"
        "Analyze the image and answer based on both context and query."
    )
])

In [19]:
rag_chain = (
    RunnableMap({
        "context": (lambda x: retriever.invoke(x["query"])),  # retriever only gets query text
        "query": (lambda x: x["query"]),                     # pass query along
        "image": (lambda x: x["image"])                      # pass image along
    })
    | prompt
    | model
)

In [20]:
result = rag_chain.invoke({
    "query": "can you give me a detail of nike present in the image and also provide the store location?",
    "image": {
        "type": "image_url",
        "image_url": "https://static.nike.com/a/images/t_PDP_1728_v1/f_auto,q_auto:eco/1705ca64-fbc8-4b79-a451-4ab77760c219/dunk-low-older-shoes-C7T1cx.png"
    }
})

In [21]:
display(Markdown(result.content))

The image displays Nike Air Max Plus sneakers. These sneakers feature a brown upper with a black Nike Swoosh logo on the side and a visible Air Max unit in the heel, all on a white sole.

The store location for these Nike Air Max Plus sneakers is Kings Way, Kathmandu, Nepal.